## How many correct trading signals (up/down) are needed to generate alpha on the S&P 500?

In [1]:
# Import Libaries
import workshop_library as wl
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (20.0, 8.0)
fontsize=20
import numpy as np
from workshop_library import financial_backtest, ws_helper
import ipywidgets
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

In [2]:
ticker = 'SPY' # Choose from SPY GLD VIX AAPL MSFT VXX XIV EURUSD N225
# Load SP 500 Data
df = wl.get_symbol(ticker)

In [3]:
hover = HoverTool(
    tooltips=[
    ("Date", "$x{%F}"),
    ("value", "$y{1.2f}"),
    ]
    , 
    formatters = 
    {
        '$x': 'datetime'
    }
    , mode='vline'
)

In [4]:
s1 = figure(plot_width=950, plot_height=400, x_axis_type="datetime", toolbar_location = 'above', 
           tools = ['crosshair', 'box_zoom', 'reset', 'save', 'undo', hover], 
           title = '%s Price Development' % ticker)

s1.xaxis.axis_label_text_font_size = "40pt"
s1.yaxis.axis_label_text_font_size = "40pt"

percentage_of_correct = .53
cost_per_trade_in_bp = 2
risk_free_interest = .02

df.index = pd.to_datetime(df.index)
df['target'] = df['Close'].pct_change().shift(-1) > 0
df['Signal'] = df['target']
rotated = (~df.target.sample(frac = 1-percentage_of_correct).astype(bool))*1
df.loc[rotated.index, 'Signal'] = rotated
fb = financial_backtest.Backtest(trading_signal=df['Signal'], underlying_series=df['Close'], transaction_cost_in_bp=cost_per_trade_in_bp)
y1 = fb.performance
x = fb.performance.index
p = s1.line(x = x , y = y1, color='navy', alpha=1, legend = 'Strategy Performance', line_width = 1.5)
s1.line(x = x, y = pd.Series(1+df.Close.pct_change().fillna(0)).cumprod()*100, legend = 'Underlying', line_width = 1.5, color = 'grey')
s1.line(x = x, y =100*np.cumprod(pd.Series([np.power(1+risk_free_interest, 1/250)] * fb.performance.shape[0])), legend = 'Cash Benchmark (2 %)', line_width = 2, color = 'green')
s1.legend.location = 'top_left'
s1.legend.click_policy = 'hide'

In [5]:
def update(percentage_of_correct, cost_per_trade_in_bp):
    # Load SP 500 Data
    df = wl.get_symbol(ticker)
    df.index = pd.to_datetime(df.index)
    df['target'] = df['Close'].pct_change().shift(-1) > 0
    df['Signal'] = df['target']
    rotated = (~df.target.sample(frac = 1-percentage_of_correct).astype(bool))*1
    df.loc[rotated.index, 'Signal'] = rotated
    fb = financial_backtest.Backtest(trading_signal=df['Signal'], underlying_series=df['Close'], transaction_cost_in_bp=cost_per_trade_in_bp)
    y1 = fb.performance
    p.data_source.data['y'] = fb.performance
    #p2.data_source.data['y'] = 100*np.cumprod(pd.Series([np.power(1+risk_free_interest, 1/250)] * fb.performance.shape[0]))
    push_notebook()
    print ('sharpe ratio: %1.2f' % fb.sharpe_ratio)

In [6]:
style = {'description_width': 'initial'}
f = ipywidgets.interact_manual(
    update, 
    percentage_of_correct = ipywidgets.FloatSlider(min=.5, max=1, step=.01, value = 0.53, style=style), 
    cost_per_trade_in_bp = ipywidgets.FloatSlider(min=0,max=5, step=.25, value = 2, style=style))
    
    #risk_free_interest = ipywidgets.FloatSlider(min=0,max=5, step=.1, value=2, style=style))


interactive(children=(FloatSlider(value=0.53, description='percentage_of_correct', max=1.0, min=0.5, step=0.01, style=SliderStyle(description_width='initial')), FloatSlider(value=2.0, description='cost_per_trade_in_bp', max=5.0, step=0.25, style=SliderStyle(description_width='initial')), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

In [7]:
show(s1, notebook_handle=True);